# Load trained model
Please ensure that you have completed **'train'** which means you have 'gesture_model.pth' before running the following code.

In [1]:
import torch
import torchvision

In [3]:
model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 3)

In [7]:
model.load_state_dict(torch.load('gesture_model.pth'))

<All keys matched successfully>

In [8]:
# Operating acceleration with GPU.
device = torch.device('cuda')
model = model.to(device)

# Preprocessing
The model has been imported previously. Next, the image from the camera needs to be processed to make it identical to the image used for training the model. The steps are as follows:

1. BGR-->RGB
2. HWC-->CHW
3. Standardize with the same parameters as during training (camera provides values in the range of [0-255] and trains the loaded image in the range of [0,1], so we need to scale 255.0
4. Put data into GPU
5. Add dimension

In [9]:
import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

# Display the picture

In [10]:
import traitlets
from IPython.display import display
import ipywidgets
import ipywidgets.widgets as widgets
from camera import Camera
from image import bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)

blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')
#prediction_widget = ipywidgets.Text(description='prediction')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(widgets.HBox([image]))

Next, create a function that will be called whenever the camera value changes. This function performs the following steps

1. Pre process camera images
2. Executive neural network
3. When the output of the neural network indicates that we are blocked, we will assign a value to 'a', otherwise we will continue to move forward.

In [11]:
import torch.nn.functional as F
import time
import sys

a=0
one_blocked=0.0
two_blocked=0.0
three_blocked=0.0

def update(change):
    global one_blocked, two_blocked, three_blocked,a
    x = change['new'] 
    x = preprocess(x)
    y = model(x)
    
# apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
#  y = F.softmax(y, dim=1).detach().cpu().numpy().flatten()
    y = F.softmax(y, dim=1)
    one_blocked = float(y.flatten()[0])
    two_blocked = float(y.flatten()[1])
    three_blocked = float(y.flatten()[2])
    if(one_blocked > two_blocked and one_blocked > three_blocked):
        a = 0
    elif(two_blocked > one_blocked and two_blocked > three_blocked):
        a = 1
    elif(three_blocked > one_blocked and three_blocked > one_blocked):
        a = 2


    
    '''index = y.argmax()
    if y[index]>0.70:
        prediction_widget.value = hand[index]
    else:
        prediction_widget.value = hand[0]'''


    time.sleep(0.001)

update({'new': camera.value})  # call the function once to intialize

In [12]:
# Using camera.observe() to dynamically connected with the previously defined update ()
camera.observe(update, names='value')  

After running the cell code block above, start generating new commands for each detected photo.

If you want to stop this, you can cancel it by executing the following code.

# Game Start!

In [13]:
### import colors
from colorama import Fore, Back, Style
import random
display(widgets.HBox([image]))
def name_of_value(val):
    if val == 0:
        return "Rock    ";
    if val == 1:
        return "Paper   ";
    if val == 2:
        return "Scissor ";
    
# main process
game_count = won_count = 0
TIME_DELTA = 0.7
try:
    while True: # forever loop
        # wait for signal
        sys.stdout.write("\n\rAre you ready?") 
        time.sleep(2.0)
        #GPIO.wait_for_edge(BUTTON_PIN, GPIO.RISING)
        
        # reset light and rotation
        game_count = game_count+1
        sys.stdout.flush()
        sys.stdout.write("\rGame %2s: Rock-" % game_count)
        time.sleep( TIME_DELTA )    
        
        # Rock-
        sys.stdout.flush()        
        sys.stdout.write("\rGame %2s: Paper-" % game_count)
        time.sleep( TIME_DELTA )    
        
        # Paper-
        sys.stdout.flush()
        sys.stdout.write("\rGame %2s: Scissors-" % game_count)
        time.sleep( TIME_DELTA )
              
        # Scissors (GO!)
        sys.stdout.flush()
        sys.stdout.write("\rGame %2s: GO!" % game_count )
        
        rint = random.randint(0,2)
              
        # Wait a little and detect hand gesture
        time.sleep( TIME_DELTA )
        sint = a; 
        sys.stdout.write("\rGame %2s  " % game_count)
        sys.stdout.write("Opponent: %s You: %s " % (name_of_value(rint), name_of_value(sint)))  
        sys.stdout.write("\tR:%f, P:%f, S:%f\t" % (one_blocked, two_blocked, three_blocked) )
        
        # win or lose
        diff = 0
        diff = (rint - sint) % 3;
        if diff == 0:
            sys.stdout.write(Fore.YELLOW+"Draw")
            #tie_game();
        elif diff == 1:
            sys.stdout.write(Fore.RED+"You lose")
            #won_game();
        elif diff == 2:
            sys.stdout.write(Fore.GREEN + "You win!!")
            won_count = won_count + 1
            #lost_game();
        sys.stdout.write(Style.RESET_ALL)
        
except KeyboardInterrupt:
    sys.stdout.write(Style.RESET_ALL + "\n")
    # statistics at the end
    if game_count != 0:
        print("Toatal score: %s/%s\n" % (won_count, game_count) )
    sys.exit(0)


Game  1  Opponent: Scissor  You: Rock     	R:0.999966, P:0.000001, S:0.000034	You win!!
Game  2  Opponent: Scissor  You: Rock     	R:0.999941, P:0.000000, S:0.000058	You win!!
Game  3  Opponent: Paper    You: Rock     	R:0.999902, P:0.000001, S:0.000096	You lose
Game  4  Opponent: Scissor  You: Rock     	R:0.999100, P:0.000007, S:0.000893	You win!!
Game  5  Opponent: Paper    You: Paper    	R:0.000018, P:0.996413, S:0.003569	Draw
Game  6  Opponent: Rock     You: Rock     	R:0.999871, P:0.000000, S:0.000129	Draw
Game  7  Opponent: Paper    You: Rock     	R:0.999835, P:0.000007, S:0.000157	You lose
Game  8  Opponent: Scissor  You: Rock     	R:0.995897, P:0.000876, S:0.003226	You win!!
Game  9  Opponent: Rock     You: Rock     	R:0.999302, P:0.000008, S:0.000690	Draw
Game 10  Opponent: Paper    You: Rock     	R:0.995800, P:0.000000, S:0.004199	You lose
Game 11  Opponent: Scissor  You: Paper    	R:0.000076, P:0.982635, S:0.017289	You lose
Game 12  Opponent: Rock     You: Rock     	R:0.999

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Game 22  Opponent: Scissor  You: Rock     	R:0.999842, P:0.000058, S:0.000101	You win!!
Are you ready?
Toatal score: 9/22

Traceback (most recent call last):
  File "<ipython-input-13-273e9e34f767>", line 20, in <module>
    time.sleep(2.0)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-273e9e34f767>", line 72, in <module>
    sys.exit(0)
SystemExit: 0

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/ultratb.py", line 1169, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/ultratb.py", line 316, in wrapped
    retur

TypeError: object of type 'NoneType' has no len()

In [10]:
camera.unobserve(update, names='value')
time.sleep(1)
#robot.stop()

In [11]:
camera_link.unlink()  # don't stream to browser (will still run camera)

In [ ]:
camera_link.link()  # stream to browser (wont run camera)